Imports

In [91]:
import pandas as pd

Access files from this path: "C:\Users\Daniel Price\Documents\Documents\Exeter\Data_Science_in_Economics\Empirical_Project\Data"

for each file, from 2000-01 to 2021-22 (inclusive), do the following:
Convert the csv file to a pandas dataframe
Since each year is the same data, but for a different year, we can concatenate the dataframes together.

In [92]:
import os
import pandas as pd
import re

# Function to parse dates adaptively
def parse_dates(dates):
    # Checking the year format based on length and adjusting accordingly
    if len(dates[0].split('/')[-1]) == 2:
        return pd.to_datetime(dates, format='%d/%m/%y')
    elif len(dates[0].split('/')[-1]) == 4:
        return pd.to_datetime(dates, format='%d/%m/%Y')
    else:
        # Fallback to infer the format if unexpected length
        return pd.to_datetime(dates, infer_datetime_format=True)

# Data directory
data = r"C:\Users\Daniel Price\Documents\Documents\Exeter\Data_Science_in_Economics\Empirical_Project\Data"

# Get the list of files in the data directory
file_list = os.listdir(data)

# Initialize an empty dataframe to store the concatenated data
concatenated_df = pd.DataFrame()

# Define the pattern for the file names
pattern = re.compile(r'^\d{4}-\d{2}\.csv$')

# Initialize a counter for the number of files converted
file_count = 0

# Iterate over each file
for file_name in file_list:
    # Check if the file matches the pattern for a CSV file
    if pattern.match(file_name):
        # Construct the file path
        file_path = os.path.join(data, file_name)
        
        # Read the CSV file into a dataframe, explicitly parsing the date
        df = pd.read_csv(file_path, parse_dates=['Date'], date_parser=parse_dates)
        
        # Concatenate the dataframe to the existing data
        concatenated_df = pd.concat([concatenated_df, df])

        # Increment the counter
        file_count += 1

# Print the concatenated dataframe
print(concatenated_df)

# Print the number of files converted
print(f"Number of files converted: {file_count}")


The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
The argument 'date_parser' is deprecated and will be removed in a future version

    Div       Date     HomeTeam  ... MaxCAHA  AvgCAHH  AvgCAHA
0    E0 2000-08-19     Charlton  ...     NaN      NaN      NaN
1    E0 2000-08-19      Chelsea  ...     NaN      NaN      NaN
2    E0 2000-08-19     Coventry  ...     NaN      NaN      NaN
3    E0 2000-08-19        Derby  ...     NaN      NaN      NaN
4    E0 2000-08-19        Leeds  ...     NaN      NaN      NaN
..   ..        ...          ...  ...     ...      ...      ...
318  E0 2024-04-13  Bournemouth  ...    1.84     2.08     1.79
319  E0 2024-04-14    Liverpool  ...    2.06     1.87     2.00
320  E0 2024-04-14     West Ham  ...    1.83     2.10     1.79
321  E0 2024-04-14      Arsenal  ...    2.02     1.91     1.95
322  E0 2024-04-15      Chelsea  ...    2.09     1.85     2.02

[9063 rows x 146 columns]
Number of files converted: 24


VAR was implemented after in the 2019/20 season, so we will mark the data which had VAR with a 1, and the data which did not have VAR with a 0. We will make a new column called 'VAR' which will have this information.

In [93]:
# Convert 'Date' column to datetime format, handling unparseable dates by setting them as NaT
concatenated_df['Date'] = pd.to_datetime(concatenated_df['Date'], format='%d/%m/%Y', errors='coerce')

# Identify rows where the date conversion resulted in NaT
nat_dates = concatenated_df[concatenated_df['Date'].isna()]

# Print the count of NaT entries
print(f"Number of unparseable dates set as NaT: {nat_dates.shape[0]}")

# If there are any NaT values, print them out
if not nat_dates.empty:
    print("List of unparseable dates:")
    print(nat_dates['Date'])  # This will only show NaT, for actual date strings causing issues, you should print another column if available

# Define the date for VAR implementation
var_start_date = pd.Timestamp('2019-06-01')

# Add a 'VAR' column based on the year extracted from the 'date'. VAR implemented from 2019 onwards.
concatenated_df['VAR'] = concatenated_df['Date'].apply(lambda x: 1 if x >= var_start_date else 0)

# Print the concatenated dataframe with the new 'VAR' column
print(concatenated_df)

# Print the number of matches listed as having VAR
print(f"Number of matches listed as having VAR: {concatenated_df['VAR'].sum()}")

# Write the concatenated dataframe to a CSV file in the same directory as the datasets
data_path = 'your/directory/path'  # Adjust this to your correct path
concatenated_df.to_csv(os.path.join(data, 'combined_data.csv'), index=False)

Number of unparseable dates set as NaT: 0
    Div       Date     HomeTeam        AwayTeam  ...  MaxCAHA  AvgCAHH AvgCAHA  VAR
0    E0 2000-08-19     Charlton        Man City  ...      NaN      NaN     NaN    0
1    E0 2000-08-19      Chelsea        West Ham  ...      NaN      NaN     NaN    0
2    E0 2000-08-19     Coventry   Middlesbrough  ...      NaN      NaN     NaN    0
3    E0 2000-08-19        Derby     Southampton  ...      NaN      NaN     NaN    0
4    E0 2000-08-19        Leeds         Everton  ...      NaN      NaN     NaN    0
..   ..        ...          ...             ...  ...      ...      ...     ...  ...
318  E0 2024-04-13  Bournemouth      Man United  ...     1.84     2.08    1.79    1
319  E0 2024-04-14    Liverpool  Crystal Palace  ...     2.06     1.87    2.00    1
320  E0 2024-04-14     West Ham          Fulham  ...     1.83     2.10    1.79    1
321  E0 2024-04-14      Arsenal     Aston Villa  ...     2.02     1.91    1.95    1
322  E0 2024-04-15      Chelsea   

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`


Check for columns with missing data

In [94]:
# Check for columns without NaN values
no_nan_columns = concatenated_df.columns[concatenated_df.notna().all()].tolist()

print("Columns without NaN values:", no_nan_columns)

Columns without NaN values: ['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HC', 'AC', 'HF', 'AF', 'HY', 'AY', 'HR', 'AR', 'VAR']


Causal Forest
We will use the causal forest model to show the effects of VAR on the number of goals scored in a match.

In [95]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from econml.dml import CausalForestDML

# Assuming 'concatenated_df' is your DataFrame and is already loaded
# Preprocess Date column
concatenated_df['Date'] = pd.to_datetime(concatenated_df['Date'])
concatenated_df['Year'] = concatenated_df['Date'].dt.year
concatenated_df['Month'] = concatenated_df['Date'].dt.month
concatenated_df['Day'] = concatenated_df['Date'].dt.day

# Selecting and preparing data
features = ['Year', 'Month', 'Day', 'HTHG', 'HTAG',
            'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 
            'AY', 'HR', 'AR']
X = concatenated_df[features]
Y = concatenated_df['FTHG'] + concatenated_df['FTAG']
T = concatenated_df['VAR']



# Define and configure the causal forest model
causal_forest = CausalForestDML(
    model_y=GradientBoostingRegressor(),  # Model for the outcome
    model_t=GradientBoostingClassifier(),  # Model for the treatment
    discrete_treatment=True,
    random_state=123
)

# Fit the causal forest model
causal_forest.fit(Y=Y, T=T, X=X)

# How much the total number of goals is expected to change due to VAR for each match
effects = causal_forest.effect(X)
print("Predicted effect of VAR on the total number of goals scored in the match: ", effects[:10])

# Estimate of VAR's effect on goals scored in a match
ate = np.mean(effects)
print("Estimated effect of VAR on the total number of goals scored in the match: ", ate)


DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
Not all column names are strings. Coercing to strings for now.


Predicted effect of VAR on the total number of goals scored in the match:  [ 2171.8388942  -3164.37744341     5.10152737  -804.6930611
  3210.85307572  -266.39283789  2785.18991734   106.42579257
  -199.66917394  1319.90337324]
Estimated effect of VAR on the total number of goals scored in the match:  -6.355027773457843


In [96]:
# Convert effects to a DataFrame
effects_df = pd.DataFrame(effects, columns=['VAR_Effect'])

# Concatenate this DataFrame with the original match data DataFrame
# Ensure that the indices align correctly
concatenated_df['VAR_Effect'] = effects_df['VAR_Effect']

# Now, you can see the VAR effect alongside all other match data
print(concatenated_df[['Date', 'HomeTeam', 'AwayTeam', 'VAR_Effect']].head(10))


        Date    HomeTeam       AwayTeam   VAR_Effect
0 2000-08-19    Charlton       Man City  2171.838894
1 2000-08-19     Chelsea       West Ham -3164.377443
2 2000-08-19    Coventry  Middlesbrough     5.101527
3 2000-08-19       Derby    Southampton  -804.693061
4 2000-08-19       Leeds        Everton  3210.853076
5 2000-08-19   Leicester    Aston Villa  -266.392838
6 2000-08-19   Liverpool       Bradford  2785.189917
7 2000-08-19  Sunderland        Arsenal   106.425793
8 2000-08-19   Tottenham        Ipswich  -199.669174
9 2000-08-20  Man United      Newcastle  1319.903373


DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`


In [97]:
#sorting to find matches with the highest VAR effects
sorted_df = concatenated_df.sort_values(by='VAR_Effect', ascending=False)
print(sorted_df[['Date', 'HomeTeam', 'AwayTeam', 'VAR_Effect']].head(10))

          Date     HomeTeam     AwayTeam   VAR_Effect
218 2020-01-12  Aston Villa     Man City  7538.839456
218 2017-01-22      Chelsea         Hull  7538.839456
218 2016-01-17        Stoke      Arsenal  7538.839456
218 2021-02-06       Fulham     West Ham  7538.839456
218 2006-01-21    Newcastle    Blackburn  7538.839456
218 2015-01-18     West Ham         Hull  7538.839456
218 2002-01-19        Derby      Ipswich  7538.839456
218 2014-01-19      Swansea    Tottenham  7538.839456
218 2007-01-13    Blackburn      Arsenal  7538.839456
218 2013-01-16      Chelsea  Southampton  7538.839456


Second Causal Forest

In [98]:
import pandas as pd
from dowhy import CausalModel
from IPython.display import Image, display

# Assuming 'concatenated_df' is your DataFrame and is already loaded
# Preprocess Date column


ModuleNotFoundError: No module named 'dowhy'